In [3]:
import pandas as pd
import numpy as np


In [4]:
content_data = pd.read_csv("cleared_content.csv")
topics_data = pd.read_csv("cleared_topics.csv")

In [5]:
# replace NaN values with an empty string
content_data['title'].fillna('', inplace=True)
content_data['description'].fillna('', inplace=True)
content_data['text'].fillna('', inplace=True)
content_data['kind'].fillna('', inplace=True)

topics_data['title'].fillna('', inplace=True)
topics_data['description'].fillna('', inplace=True)

topics representation (title, description, and title of it's parent, and grandparent)

In [8]:
def generate_combined_text(topics_data):
    combined_texts = []

    for index, row in topics_data.iterrows():
        child = row
        parent_id = child['parent']

        child_text = f"{child['title']} {child['description']}"

        if parent_id != 'NaN':
            parent = topics_data[topics_data['id'] == parent_id]

            if not parent.empty:
                parent_title = parent['title'].values[0]
                child_text += f" {parent_title}"

                grandparent_id = parent['parent'].values[0]
                if grandparent_id != 'NaN':
                    grandparent = topics_data[topics_data['id'] == grandparent_id]

                    if not grandparent.empty:
                        grandparent_title = grandparent['title'].values[0]
                        child_text += f" {grandparent_title}"
                        
                        greatgrandparent_id = grandparent['parent'].values[0]
                        if greatgrandparent_id != 'NaN':
                            greatgrandparent = topics_data[topics_data['id'] == greatgrandparent_id]

                            if not greatgrandparent.empty:
                                greatgrandparent_title = greatgrandparent['title'].values[0]
                                child_text += f" {greatgrandparent_title}"

        combined_texts.append(child_text)

    return combined_texts

In [9]:
topics_data['combined_text'] = generate_combined_text(topics_data)

In [10]:
topics_data

,id,title,description,channel,category,level,language,parent,has_content,combined_text
0,t_00004da3a1b2,detect resistor,investig materi caus reduct deflect connect se...,000cf7,source,4,bg,t_16e29365b50d,True,detect resistor investig materi caus reduct de...
1,t_000095e03056,unit enlarg similar,,b3f329,aligned,2,en,t_aa32fb6252dc,False,unit enlarg similar junior high level 3 ghana...
2,t_00068291e9a4,input output function,understand littl function,8e286a,source,4,pt,t_d14b6c2a2b70,True,input output function understand littl functio...
3,t_00069b63a70a,transcript,,6e3ba4,source,3,en,t_4054df11a74e,True,transcript flow chart logic think engin blossom
4,t_0006d41a73a8,graph exponenti function algebra 2 level,learn graph complex exponenti function form,000cf7,source,4,bg,t_e2452e21d252,True,graph exponenti function algebra 2 level learn...
...,...,...,...,...,...,...,...,...,...,...
76967,t_fffb0bf2801d,graph function,,e77b55,aligned,4,en,t_676e6a1a4dc7,False,graph function unit 4 relat function mathemat...
76968,t_fffbe1d5d43c,inscrib shape problem solv,use properti inscrib angl prove properti inscr...,0c929f,source,4,sw,t_50145b9bab3f,True,inscrib shape problem solv use properti inscri...
76969,t_fffe14f1be1e,lesson 7,,6e90a7,aligned,6,es,t_d448c707984d,True,lesson 7 unit 4 spanish activ 1st 01 first
76970,t_fffe811a6da9,determin relationship polar coordin cartesian ...,5 bassakh 1st floor 0499 yasf,9fd860,source,2,ar,t_5b4f3ba4eb7d,True,determin relationship polar coordin cartesian ...


content representation

In [11]:
def preprocess_content_text(content_data):
    content_text = content_data['title'] + ' ' + np.where(
        (content_data['description'] == ''),
        content_data['text'] + ' ' + content_data['kind'],
        content_data['description'] + ' ' + content_data['kind']
    )
    return content_text

In [12]:
content_data['combined_text'] = preprocess_content_text(content_data)

In [13]:
# keep topics where has_content is True
topics_data = topics_data[topics_data['has_content'] == True].reset_index(drop=True)

In [14]:
# save topics and content
topics_data.to_csv('all_topics_tf-idf.csv', index=False)
content_data.to_csv('all_content_tf-idf.csv', index=False)